In [ ]:
import os
os.environ["DJANGO_SETTINGS_MODULE"] = "ClusterCast.settings"
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from django.db.models.functions import Now
from asgiref.sync import sync_to_async
from tensorflow.keras.layers import Input, LSTM, Dropout, RepeatVector, TimeDistributed, Dense, Masking
from tensorflow.keras.models import Model
import ClusterPipeline.models.RNNModels as rnn
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session
from tslearn.clustering import TimeSeriesKMeans
import tensorflow as tf
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.RNNModels as rnn 
from copy import deepcopy

In [ ]:
@sync_to_async
def get_all_objects(features,steps = None):
    # Force the query to execute and load all results into memory
    params = list(cp.StockClusterGroupParams.objects.all())
    matching_groups = []
    for param in params:
        continue
        if steps is None:
            if param.cluster_features == features:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()

        else: 
            if param.cluster_features == features and param.n_steps == steps:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()
                print(group.group_params.tickers)

    group = cp.StockClusterGroup.objects.get(pk=367)
    group.load_saved_group()
    
    return matching_groups,group


cluster_features = ["close", "bb_low", "bb_high"]
steps = 20

async def create_cluster_group_params(cluster_features, steps=20):

    cluster_groups,group = await get_all_objects(features=cluster_features,steps=steps)
    return cluster_groups,group

# Run the async function

matching_groups,group = await create_cluster_group_params(cluster_features, steps=steps)

In [ ]:
@sync_to_async
def get_all_clusters(groups):
    clusters = []
    features = []
    for group in groups:
        clusters += group.clusters

    for cluster in clusters:
        models = rnn.RNNModel.objects.filter(cluster=cluster)
        for model in models: 
            if model:
                features += (model.model_features)

    features = list(set(features))
    return clusters, features

clusters, all_features = await get_all_clusters(matching_groups)

In [ ]:
test_group = group
print(test_group.id)
cluster_features = test_group.group_params.cluster_features
feature_dict = test_group.group_params.X_feature_dict
y_feature_dict = test_group.group_params.y_feature_dict
print(len(test_group.clusters))
training_dict = {}

In [ ]:
print(y_feature_dict)

In [ ]:
X_train_top, y_train_top, X_test_top, y_test_top = test_group.get_3d_array()
y_train_top_reversed = y_train_top[:, ::-1]
y_test_top_reversed = y_test_top[:, ::-1]
training_dict["first"] = (X_train_top, y_train_top, X_test_top, y_test_top)
print(X_train_top.shape)
print(y_train_top.shape)
print(X_test_top.shape)
print(y_test_top.shape)

In [ ]:
import matplotlib.pyplot as plt
def visualize_cluster_centroid(centroid, cluster_features):

    # Extract the number of features
    num_features = centroid.shape[1]

    # Set up the time steps
    time_steps = range(centroid.shape[0])

    # Plot the centroid for each feature
    for i in range(num_features):
        plt.scatter(time_steps, centroid[:,i], label=f'Feature {cluster_features[i]}')
        plt.plot(time_steps, centroid[:,i], 'kx-')  # 'kx-' for black crosses with lines

    plt.xlabel('Time Steps')
    plt.ylabel('Feature Values')
    plt.title('Centroid Visualization Over Time Steps')
    plt.legend()
    plt.show()

In [ ]:
first_alg = TimeSeriesKMeans(n_clusters=2, metric="euclidean", max_iter=5, random_state=0)
X_train_top, y_train_top, X_test_top, y_test_top = training_dict["first"]
X_train_cluster = test_group.filter_by_features(X_train_top, cluster_features, feature_dict)
X_test_cluster = test_group.filter_by_features(X_test_top, cluster_features, feature_dict)

print(X_train_cluster.shape)
print(X_test_cluster.shape)

first_alg.fit(X_train_cluster)

print(first_alg.cluster_centers_.shape)

labels = first_alg.labels_
target_label = 1


visualize_cluster_centroid(first_alg.cluster_centers_[target_label], cluster_features)
visualize_cluster_centroid(first_alg.cluster_centers_[0], cluster_features)


X_train_top = X_train_top[labels == target_label]

y_train_top = y_train_top[labels == target_label]

test_labels = first_alg.predict(X_test_cluster)
X_test_top = X_test_top[test_labels == target_label]
y_test_top = y_test_top[test_labels == target_label]

training_dict["second"] = (deepcopy(X_train_top), deepcopy(y_train_top), deepcopy(X_test_top), deepcopy(y_test_top))

In [ ]:
print(X_train_top.shape)
print(y_train_top.shape)
print(X_test_top.shape)
print(y_test_top.shape)

In [ ]:
second_alg = TimeSeriesKMeans(n_clusters=2, metric="euclidean", max_iter=5, random_state=0)
X_train_top, y_train_top, X_test_top, y_test_top = training_dict["second"]
X_train_cluster = test_group.filter_by_features(X_train_top, cluster_features, feature_dict)
X_test_cluster = test_group.filter_by_features(X_test_top, cluster_features, feature_dict)

print(X_train_cluster.shape)
print(X_test_cluster.shape)

second_alg.fit(X_train_cluster)

labels = second_alg.labels_
target_label = 1

visualize_cluster_centroid(second_alg.cluster_centers_[target_label], cluster_features)
visualize_cluster_centroid(second_alg.cluster_centers_[0], cluster_features)

print(set(labels))
X_train_top = X_train_top[labels == target_label]

y_train_top = y_train_top[labels == target_label]

test_labels = second_alg.predict(X_test_cluster)
X_test_top = X_test_top[test_labels == target_label]
y_test_top = y_test_top[test_labels == target_label]

training_dict["third"] = (deepcopy(X_train_top), deepcopy(y_train_top), deepcopy(X_test_top), deepcopy(y_test_top))

In [ ]:
print(X_train_top.shape)
print(y_train_top.shape)
print(X_test_top.shape)
print(y_test_top.shape)

In [ ]:
third_alg = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0)
X_train_top, y_train_top, X_test_top, y_test_top = training_dict["third"]
X_train_cluster = test_group.filter_by_features(X_train_top, cluster_features, feature_dict)
X_test_cluster = test_group.filter_by_features(X_test_top, cluster_features, feature_dict)

print(X_train_cluster.shape)
print(X_test_cluster.shape)

third_alg.fit(X_train_cluster)

labels = third_alg.labels_
target_label = 0

for i in range(3):
    visualize_cluster_centroid(third_alg.cluster_centers_[i], cluster_features)


print(set(labels))
X_train_top = X_train_top[labels == target_label]

y_train_top = y_train_top[labels == target_label]

test_labels = third_alg.predict(X_test_cluster)
X_test_top = X_test_top[test_labels == target_label]
y_test_top = y_test_top[test_labels == target_label]

training_dict["last"] = (deepcopy(X_train_top), deepcopy(y_train_top), deepcopy(X_test_top), deepcopy(y_test_top))

In [ ]:
print(X_train_top.shape)
print(y_train_top.shape)
print(X_test_top.shape)
print(y_test_top.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, TimeDistributed, Dense, Concatenate, Permute, Reshape, Multiply
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.layers import Layer
from keras.layers import Activation, Flatten
from tensorflow.keras.regularizers import L1, L2, L1L2

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def create_model(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # masking_layer = Masking(mask_value=0.0, name='masking_layer')(input_layer)

    # Encoder

    encoder_lstm2 = LSTM(units=100, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(input_layer)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=50, activation='tanh', return_sequences=False, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=False, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Repeat Vector
    repeat_vector = RepeatVector(latent_dim, name='repeat_vector')(encoder_dropout3)

    # Decoder
    decoder_lstm1 = LSTM(units=100, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(repeat_vector)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=50, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout2)


    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=time_distributed_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mae")

    return model_lstm



In [ ]:
def create_modelAE(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape),name = 'input_layer')

    # masking_layer = Masking(mask_value=0.0, name='masking_layer')(input_layer)

    # Encoder
    encoder_lstm1 = LSTM(units=300, activation='tanh', return_state=True,return_sequences=True,
                     name='encoder_lstm_1_freeze', kernel_regularizer=L2(.001), recurrent_regularizer=L2(.001))
    encoder_outputs1 = encoder_lstm1(input_layer)
    encoder_states1 = encoder_outputs1[1:]

    encoder_lstm2 = LSTM(units=200, activation='tanh', return_state=True,return_sequences=True, name = 'encoder_lstm_2_freeze',
                         )
    encoder_outputs2 = encoder_lstm2(encoder_outputs1[0])
    encoder_states2 = encoder_outputs2[1:]

    encoder_lstm3 = LSTM(units=150, activation='tanh', return_state=True,return_sequences=True, name='encoder_lstm_3_freeze')
    encoder_outputs3 = encoder_lstm3(encoder_outputs2[0])
    encoder_states3 = encoder_outputs3[1:]

    encoder_lstm4 = LSTM(units=100, activation='tanh', return_state=True,return_sequences=True, name='encoder_lstm_4_restore')
    encoder_outputs4 = encoder_lstm4(encoder_outputs3[0])
    encoder_states4 = encoder_outputs4[1:]


    # attention = Dense(1, activation='tanh')(encoder_lstm4)
    # attention = Flatten()(attention)
    # attention_weights = Activation('softmax')(attention)
    # context = Multiply()([encoder_lstm4, Permute([2, 1])(RepeatVector(6)(attention_weights))])

    decoder_inputs = RepeatVector(21, name='repeat_vector')(encoder_states4[0])
    

    # Decoder
    decoder_lstm1 = LSTM(units=300, activation='tanh', return_sequences=True, name='decoder_lstm_1_freeze',
    
                        )(decoder_inputs, initial_state=encoder_states1)
    decoder_lstm2 = LSTM(units=200, activation='tanh', return_sequences=True, name='decoder_lstm_2_freeze',
                        )(decoder_lstm1, initial_state=encoder_states2)
    decoder_lstm3 = LSTM(units=150, activation='tanh', return_sequences=True, name='decoder_lstm_3_freeze',
                         )(decoder_lstm2, initial_state=encoder_states3)
    decoder_lstm4 = LSTM(units=100, activation='tanh', return_sequences=True, name='decoder_lstm_4_restore',
                         )(decoder_lstm3, initial_state=encoder_states4)

    # decoder_lstm3 = LSTM(units=5, activation='tanh', return_sequences=True, name='decoder_lstm_3_restore',
    #                      )(decoder_dropout2)
    # decoder_dropout3 = Dropout(0.2, name='decoder_dropout_3_restore')(decoder_lstm3)

    

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_lstm4)

    # final_output = time_distributed_output[:, -6:, :]

    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=time_distributed_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mae")

    return model_lstm

In [ ]:

# Custom Attention Layer

def create_model_with_attention2(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape),name = 'input_layer')

    # Encoder

    encoder_lstm1 = LSTM(units=40, activation='tanh', return_sequences=True, name='encoder_lstm_1_freeze',kernel_regularizer=L2(.0001), recurrent_regularizer=L2(.0001))(input_layer)
    encoder_dropout1 = Dropout(0.2, name='encoder_dropout_1_freeze')(encoder_lstm1)

    encoder_lstm2 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_2_freeze')(encoder_dropout1)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_freeze')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=25, activation='tanh', return_sequences=True, name='encoder_lstm_3_freeze')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_freeze')(encoder_lstm3)

    output_lstm = LSTM(units=6, activation='tanh', return_sequences=True, name='outputLSTM_freeze')(encoder_dropout3)
    encoder_output = Dropout(0.2, name='encoder_output_freeze')(output_lstm)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Attention Layer
    # attention = AttentionLayer(name='attention_layer')(encoder_dropout4)
     # Attention Mechanism
    attention = Dense(1, activation='tanh')(encoder_output)
    attention = Flatten()(attention)
    attention_weights = Activation('softmax')(attention)
    context = Multiply()([encoder_output, Permute([2, 1])(RepeatVector(6)(attention_weights))])

    # Decoder
    decoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='decoder_lstm_1')(context)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='decoder_lstm_2')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2')(decoder_lstm2)

    # decoder_lstm3 = LSTM(units=5, activation='tanh', return_sequences=True, name='decoder_lstm_3_restore')(decoder_dropout2)
    # decoder_dropout3 = Dropout(0.2, name='decoder_dropout_3_restore')(decoder_lstm3)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout2)

    # final_output = time_distributed_output[:, -6:, :]

    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=final_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mae")

    return model_lstm


In [ ]:
def attention_mechanism(encoder_outputs, decoder_state):
    # Assuming encoder_outputs is [batch_size, input_steps, features]
    # and decoder_state is [batch_size, features]
    score = Dense(encoder_outputs.shape[2])(decoder_state)  # Project decoder state
    score = tf.expand_dims(score, 1)  # Expand dims to add input_steps axis
    score = score + encoder_outputs  # Add to encoder outputs
    attention_weights = Activation('softmax')(score)  # Compute attention weights
    context_vector = tf.reduce_sum(attention_weights * encoder_outputs, axis=1)
    return context_vector

def create_autoencoder(input_steps, output_steps, features):
    # Encoder

    encoder_inputs = Input(shape=(input_steps, features))

    encoder_lstm1 = LSTM(200, return_sequences=True, kernel_regularizer=L2(.001), recurrent_regularizer=L2(.001))
    encoder_output1 = encoder_lstm1(encoder_inputs)

    encoder_lstm_final = LSTM(100, return_state=True, return_sequences=True)
    encoder_outputs, state_h, state_c = encoder_lstm_final(encoder_output1)

    # Decoder
    decoder_initial_input = RepeatVector(output_steps)(state_h)  # Prepare decoder inputs


    decoder_lstm = LSTM(100, return_sequences=True)
    decoder_output1 = decoder_lstm(decoder_initial_input, initial_state=[state_h, state_c])

    # Manually apply attention mechanism for each timestep
    context_vectors_list1 = []
    for t in range(output_steps):
        
        # Apply attention mechanism
        context_vector_t1 = attention_mechanism(encoder_outputs, decoder_output1[:, t, :])
        context_vectors_list1.append(context_vector_t1)

    # Concatenate the list of context vectors
    context_vectors = tf.stack(context_vectors_list1, axis=1)

    # Concatenate context vectors with decoder outputs
    decoder_combined_context1 = Concatenate(axis=-1)([context_vectors, decoder_output1])

    decoder_lstm2 = LSTM(50, return_sequences=True)
    decoder_output2 = decoder_lstm2(decoder_combined_context1)

    # Manually apply attention mechanism for each timestep
    context_vectors_list2 = []
    for t in range(output_steps):
        
        # Apply attention mechanism
        context_vector_t2 = attention_mechanism(encoder_outputs, decoder_output2[:, t, :])
        context_vectors_list2.append(context_vector_t2)
    
    # Concatenate the list of context vectors
    context_vectors2 = tf.stack(context_vectors_list2, axis=1)
    decoder_combined_context2 = Concatenate(axis=-1)([context_vectors2, decoder_output2])



    # Output layer for reconstruction
    output = TimeDistributed(Dense(1))(decoder_combined_context2)

    # Create and compile the model
    model = Model(inputs=encoder_inputs, outputs=output)
    model.compile(optimizer='adam', loss='mse')  # Use appropriate loss

    return model


In [ ]:
import tensorflow as tf

def custom_loss_function(y_true, y_pred, past_steps, future_weight):
    """
    Custom loss function that assigns different weights to the errors in predicting 
    past and future values in a sequence.

    Parameters:
    y_true (tensor): The true values.
    y_pred (tensor): The predicted values from the model.
    past_steps (int): The number of steps in the sequence corresponding to past values.
    future_steps (int): The number of steps in the sequence corresponding to future values.
    future_weight (float): The weight to assign to the errors in the future values.

    Returns:
    tensor: The computed weighted loss.
    """
    # Split the true and predicted values into past and future parts
    y_true_past, y_true_future = y_true[:, :past_steps], y_true[:, past_steps:]
    y_pred_past, y_pred_future = y_pred[:, :past_steps], y_pred[:, past_steps:]

    # Calculate mean absolute error for past and future parts
    past_loss = tf.keras.losses.mean_squared_error(y_true_past, y_pred_past)
    future_loss = tf.keras.losses.mean_squared_error(y_true_future, y_pred_future)

    if future_weight == 1:
        # If future_weight is 1, calculate normal MAE across the entire sequence
        total_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    else:
        # Weight the future loss and combine it with the past loss
        weighted_future_loss = future_loss * future_weight
        total_loss = tf.reduce_mean(past_loss + weighted_future_loss)

    return total_loss


In [ ]:
def filter_by_features(sequence, features,X_feature_dict):
    sequence = deepcopy(sequence)
    feature_indices = [X_feature_dict[feature] for feature in features]
    return sequence[:,:,feature_indices]

def filter_y_by_features(sequence, features,y_feature_dict):
    sequence = deepcopy(sequence)
    feature_indices = [y_feature_dict[feature] for feature in features]
    print(feature_indices)
    return sequence[:,feature_indices]

In [241]:
import random
import numpy as np
# features = random.sample(all_features, 30)
features = [
    "sumpctChg+pctChgsma30Close_4", "sumpctChg+pctChgema10Close_5", "pctChg+pctChgsma5Close", 
    "sumpctChg+sma10Close_6", "sumpctChg+pctChgsma50Close_5", "sumpctChg+bb_maClose_6", 
    "sumpctChglow_1", "sumpctChg+ema100Close_5", "sumpctChg+ema10Close_3", "sumpctChg+pctChgsma5Close_2", 
    "pctChglow", "sumpctChgvolume_3", "sumpctChgema100_3", "sumpctChgsma10_4", "sumpctChgsma10_2", 
    "sumpctChg+ema30Close_1", "pctChgema50", "sumpctChgsma5_2", "sumpctChg+pctChgema5Close_1", "pctChghigh", 
    "pctChg+sma30Close", "pctChgVix", "sumpctChgema20_3", "sumpctChg+bb_maClose_4", "sumpctChg+ema50Close_5", 
    "sumpctChg+sma20Close_5", "pctChg+pctChgsma100Close", "sumpctChgema20_2", "sumpctChg+pctChgema5Close_5", 
    "pctChgema30", "sumpctChg+sma5Close_6", "sumpctChg+ema50Close_3", "sumpctChg+pctChgema30Close_1", 
    "sumpctChgema30_4", "sumpctChgema5_3", 'pctChgclose', 'pctChgema10', 'pctChgema20', 'pctChgema100',
]
# target_features = ['pctChgclose-3','pctChgclose-2','pctChgclose-1','pctChgclose-0','pctChgclose+1','pctChgclose+2','pctChgclose+3','pctChgclose+4','pctChgclose+5','pctChgclose+6']
# target_features = ['pctChgclose+1','pctChgclose+2']
# target_features = ['sumPctChgclose+1', 'sumPctChgclose+2', 'sumPctChgclose+3', 'sumPctChgclose+4', 'sumPctChgclose+5', 'sumPctChgclose+6']
target_features = ['pctChgclose-14', 'pctChgclose-13', 'pctChgclose-12', 'pctChgclose-11', 'pctChgclose-10', 'pctChgclose-9', 'pctChgclose-8', 'pctChgclose-7', 'pctChgclose-6', 'pctChgclose-5', 'pctChgclose-4', 'pctChgclose-3', 'pctChgclose-2', 'pctChgclose-1', 'pctChgclose-0', 'pctChgclose+1', 'pctChgclose+2', 'pctChgclose+3', 'pctChgclose+4', 'pctChgclose+5', 'pctChgclose+6']


In [ ]:
def save_decoder_initial_weights(model):
    initial_weights = {}
    for layer in model.layers:
        if 'input' in layer.name: 
            continue
        print(layer.name)
        initial_weights[layer.name] = deepcopy(layer.get_weights())
    return initial_weights

model = create_modelAE(len(features))
decoder_weights = save_decoder_initial_weights(model)

In [ ]:
# import numpy as np
# x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
# x = np.array(x)
# x = x.reshape(1,1,22)
# print(x.shape)
# indices = [12, 11, 10, 9, 8, 21, 20, 19, 18, 17, 16, 15, 14, 7, 6, 0, 1, 2, 3, 4, 5]
# print(x[:,:,indices])

In [ ]:
import numpy as np
import datetime

X_train, y_train, X_test, y_test = training_dict['first']
X_train_filtered = filter_by_features(X_train, features, feature_dict)
X_test_filtered = filter_by_features(X_test, features, feature_dict)
y_train_filtered = filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)

# learning_rates = [.0001,0.001, 0.01, 0.1]
# batch_sizes = [10, 20, 50, 100]
learning_rates = [.01]
batch_sizes = [100]


best_score = np.inf
best_lr = None
best_batch_size = None

for lr in learning_rates:
    for batch in batch_sizes:
        # Build and compile the model
        test_model = deepcopy(model)
        test_model.compile(optimizer=Adam(learning_rate=lr), loss=lambda y_true, y_pred: custom_loss_function(y_true, y_pred, 15, 1.0))

        # Callbacks
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"_lr{lr}_batch{batch}_first"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

        # Train the model
        test_model.fit(X_train_filtered, y_train_filtered, 
                  validation_data=(X_test_filtered, y_test_filtered),
                  epochs=100,  # Adjust as needed
                  batch_size=batch,
                  callbacks=[tensorboard_callback, early_stopping],
                  verbose=1)

        # Evaluate the model
        score = test_model.evaluate(X_test_filtered, y_test_filtered, verbose=1)

        # Update best score
        if score < best_score:
            best_score = score
            best_lr = lr
            best_batch_size = batch
            model = deepcopy(test_model)

        print(f"Learning Rate: {lr}, Batch Size: {batch}, Score: {score}")

# Print best configuration
print(f"Best Score: {best_score}, Learning Rate: {best_lr}, Batch Size: {best_batch_size}")

model.compile(optimizer=Adam(learning_rate=best_lr), loss='mae')
for layer in model.layers:
    if 'freeze' in layer.name and '1' in layer.name:
        print(layer.name)
        layer.trainable = False
model.save('path_to_my_model2.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('path_to_my_model2.h5', custom_objects={'custom_loss_function': lambda y_true, y_pred: custom_loss_function(y_true, y_pred, past_steps=15, future_weight=1.0)})
for layer in model.layers:
    if 'input' in layer.name:
        continue
    if '1' not in layer.name or 'restore' in layer.name:
        print("restoring weights for layer {}".format(layer.name))
        layer.set_weights(decoder_weights[layer.name])
X_train, y_train, X_test, y_test = training_dict['second']
X_train_filtered = filter_by_features(X_train, features, feature_dict)
X_test_filtered = filter_by_features(X_test, features, feature_dict)
y_train_filtered = filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)

# learning_rates = [.0001,0.001, 0.01, 0.1]
# batch_sizes = [10, 20, 50, 100]
learning_rates = [.01]
batch_sizes = [20]


best_score = np.inf
best_lr = None
best_batch_size = None

for lr in learning_rates:
    for batch in batch_sizes:
        # Build and compile the model
        test_model = deepcopy(model)
        test_model.compile(optimizer=Adam(learning_rate=lr), loss=lambda y_true, y_pred: custom_loss_function(y_true, y_pred, 15, 1.0))

        # Callbacks
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"_lr{lr}_batch{batch}_second"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

        # Train the model
        test_model.fit(X_train_filtered, y_train_filtered, 
                  validation_data=(X_test_filtered, y_test_filtered),
                  epochs=100,  # Adjust as needed
                  batch_size=batch,
                  callbacks=[tensorboard_callback, early_stopping],
                  verbose=1)

        # Evaluate the model
        score = test_model.evaluate(X_test_filtered, y_test_filtered, verbose=1)

        # Update best score
        if score < best_score:
            best_score = score
            best_lr = lr
            best_batch_size = batch
            model = deepcopy(test_model)

        print(f"Learning Rate: {lr}, Batch Size: {batch}, Score: {score}")

# Print best configuration
print(f"Best Score: {best_score}, Learning Rate: {best_lr}, Batch Size: {best_batch_size}")


In [ ]:
for layer in model.layers:
    if 'freeze' in layer.name and '2' in layer.name:
        print(layer.name)
        layer.trainable = False
model.compile(optimizer=Adam(learning_rate=best_lr), loss='mae')
model.save('path_to_my_model2.h5')

In [ ]:
model = load_model('path_to_my_model2.h5',custom_objects={'rmse': rmse})
for layer in model.layers:
    if 'input' in layer.name:
        continue
    if '2' not in layer.name and '1' not in layer.name or 'restore' in layer.name:
        layer.set_weights(decoder_weights[layer.name])
        print("restoring weights for layer {}".format(layer.name))

X_train, y_train, X_test, y_test = training_dict['third']
X_train_filtered = filter_by_features(X_train, features, feature_dict)
X_test_filtered = filter_by_features(X_test, features, feature_dict)
y_train_filtered = filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = filter_y_by_features(y_test, target_features, y_feature_dict)

# learning_rates = [.0001,0.001, 0.01, 0.1]
# batch_sizes = [10, 20, 50, 100]
learning_rates = [.01]
batch_sizes = [50]


best_score = np.inf
best_lr = None
best_batch_size = None

for lr in learning_rates:
    for batch in batch_sizes:
        # Build and compile the model
        test_model = deepcopy(model)
        test_model.compile(optimizer=Adam(learning_rate=lr), loss=lambda y_true, y_pred: custom_loss_function(y_true, y_pred, 15, 1.0))

        # Callbacks
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"_lr{lr}_batch{batch}_third"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

        # Train the model
        test_model.fit(X_train_filtered, y_train_filtered, 
                  validation_data=(X_test_filtered, y_test_filtered),
                  epochs=100,  # Adjust as needed
                  batch_size=batch,
                  callbacks=[tensorboard_callback, early_stopping],
                  verbose=1)

        # Evaluate the model
        score = test_model.evaluate(X_test_filtered, y_test_filtered, verbose=1)

        # Update best score
        if score < best_score:
            best_score = score
            best_lr = lr
            best_batch_size = batch
            model = deepcopy(test_model)

        print(f"Learning Rate: {lr}, Batch Size: {batch}, Score: {score}")

# Print best configuration
print(f"Best Score: {best_score}, Learning Rate: {best_lr}, Batch Size: {best_batch_size}")


In [ ]:
for layer in model.layers:
    if 'freeze' in layer.name and '3' in layer.name:
        print(layer.name)
        layer.trainable = False
model.compile(optimizer=Adam(learning_rate=best_lr), loss='mae')
model.save('path_to_my_model2.h5')

In [ ]:
import pandas as pd 
import numpy as np
def eval_model(X_test, y_test_old, model):

    predicted_y_old = model.predict(X_test)
    predicted_y_old = np.squeeze(predicted_y_old, axis=-1)

    print(predicted_y_old.shape)

    predicted_y_old = predicted_y_old[:,-6:]
    print(predicted_y_old.shape)
    y_test_old = y_test_old[:,-6:]
    
    predicted_y = np.cumsum(predicted_y_old, axis=1)
    y_test = np.cumsum(y_test_old, axis=1)
   

    num_days = predicted_y.shape[1]  # Assuming this is the number of days
    print(num_days)
    results = pd.DataFrame(predicted_y, columns=[f'predicted_{i+1}' for i in range(num_days)])

    for i in range(num_days):
        results[f'real_{i+1}'] = y_test[:, i]

    # Generate output string with accuracies
    output_string = f"Cluster Number:\n"
    for i in range(num_days):
        results['same_day'] = ((results[f'predicted_{i+1}'] > 0) & (results[f'real_{i+1}'] > 0)) | \
                ((results[f'predicted_{i+1}'] < 0) & (results[f'real_{i+1}'] < 0))
        accuracy = round(results['same_day'].mean() * 100,2)

        output_string += (
            f"Accuracy{i+1}D {accuracy}% "
            f"PredictedRet: {results[f'predicted_{i+1}'].mean()} "
            f"ActRet: {results[f'real_{i+1}'].mean()}\n"
        )
    
    output_string += f"Train set length: {len(X_train_filtered)} Test set length: {len(y_test)}\n"

    return output_string, results

In [ ]:
model = load_model('path_to_my_model2.h5',custom_objects={'rmse': rmse})
for layer in model.layers:
    if 'restore' in layer.name:
        print("restoring weights for layer {}".format(layer.name))
        layer.set_weights(decoder_weights[layer.name])

X_train, y_train, X_test, y_test = training_dict['last']
X_train_filtered = filter_by_features(X_train, features, feature_dict)
X_test_filtered = filter_by_features(X_test, features, feature_dict)
y_train_filtered = filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = filter_y_by_features(y_test, target_features, y_feature_dict)

# learning_rates = [.0001,0.001, 0.01, 0.1]
# batch_sizes = [10, 20, 50, 100]
learning_rates = [.01]
batch_sizes = [10]


best_score = np.inf
best_lr = None
best_batch_size = None

for lr in learning_rates:
    for batch in batch_sizes:
        # Build and compile the model
        test_model = deepcopy(model)
        test_model.compile(optimizer=Adam(learning_rate=lr), loss=lambda y_true, y_pred: custom_loss_function(y_true, y_pred, 15, 1.1))

        # Callbacks
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"_lr{lr}_batch{batch}_tuned"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=True)

        # Train the model
        test_model.fit(X_train_filtered, y_train_filtered, 
                  validation_data=(X_test_filtered, y_test_filtered),
                  epochs=250,  # Adjust as needed
                  batch_size=batch,
                  callbacks=[tensorboard_callback, early_stopping],
                  verbose=1)

        # Evaluate the model
        # score = test_model.evaluate(X_test_filtered, y_test_filtered, verbose=1)

        # Update best score
        if score < best_score:
            best_score = score
            best_lr = lr
            best_batch_size = batch
            model = deepcopy(test_model)

        print(f"Learning Rate: {lr}, Batch Size: {batch}, Score: {score}")

# Print best configuration
print(f"Best Score: {best_score}, Learning Rate: {best_lr}, Batch Size: {best_batch_size}")


# val_loss_tuned = model.evaluate(X_test_filtered, y_test_filtered)
# print(f'Validation loss: {val_loss_tuned}')

In [ ]:
fineTunedAccuracy, results_tuned = eval_model(X_test_filtered, y_test_filtered, model)

In [242]:
benchmark_model = create_autoencoder(15,len(target_features),len(features))
X_train, y_train, X_test, y_test = training_dict['last']
X_train_filtered = filter_by_features(X_train, features, feature_dict)
X_test_filtered = filter_by_features(X_test, features, feature_dict)
y_train_filtered = filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = filter_y_by_features(y_test, target_features, y_feature_dict)

print(X_train_filtered.shape)
print(y_train_filtered.shape)

# learning_rates = [.0001,0.001, 0.01, 0.1]
# batch_sizes = [10, 20, 50, 100]

learning_rates = [.001]
batch_sizes = [16]


best_score = np.inf
best_lr = None
best_batch_size = None

for lr in learning_rates:
    for batch in batch_sizes:
        # Build and compile the model
        test_model = deepcopy(benchmark_model)
        test_model.compile(optimizer=Adam(learning_rate=lr), loss='mse')

        # Callbacks
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"_lr{lr}_batch{batch}_bench"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True)

        # Train the model
        test_model.fit(X_train_filtered, y_train_filtered, 
                  validation_data=(X_test_filtered, y_test_filtered),
                  epochs=100,  # Adjust as needed
                  batch_size=batch,
                  callbacks=[tensorboard_callback, early_stopping],
                  verbose=1)

        # Evaluate the model
        # score = test_model.evaluate(X_test_filtered, y_test_filtered, verbose=1)

        # Update best score
        if score < best_score:
            best_score = score
            best_lr = lr
            best_batch_size = batch
            benchmark_model = deepcopy(test_model)

        print(f"Learning Rate: {lr}, Batch Size: {batch}, Score: {score}")

# Print best configuration
print(f"Best Score: {best_score}, Learning Rate: {best_lr}, Batch Size: {best_batch_size}")

# val_loss_benchmark = benchmark_model.evaluate(X_test_filtered, y_test_filtered)

benchmarkAccuracy, results_benchmark = eval_model(X_test_filtered, y_test_filtered, benchmark_model)

[0, 1]
[0, 1]
(314, 15, 39)
(314, 2)


2024-01-18 00:20:55.360049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-18 00:20:55.361026: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-18 00:20:55.361705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2024-01-18 00:20:56.060098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-18 00:20:56.061219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-18 00:20:56.061944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

16/20 [=======================>......] - ETA: 0s - loss: 2.6117 

...

20/20 [==============================] - 3s 29ms/step - loss: 2.4880 - val_loss: 1.8276
Epoch 2/100
20/20 [==============================] - 0s 8ms/step - loss: 1.4178 - val_loss: 1.1211
Epoch 3/100
20/20 [==============================] - 0s 8ms/step - loss: 0.9070 - val_loss: 0.7785
Epoch 4/100
20/20 [==============================] - 0s 8ms/step - loss: 0.6587 - val_loss: 0.6194
Epoch 5/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5413 - val_loss: 0.5370
Epoch 6/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4828 - val_loss: 0.4922
Epoch 7/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4390 - val_loss: 0.4762
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4258 - val_loss: 0.4475
Epoch 9/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4034 - val_loss: 0.4354
Epoch 10/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3935 - val_loss: 0.4252
Epoch 11/100
20/20 [

2024-01-18 00:21:05.755934: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-18 00:21:05.756891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-18 00:21:05.757555: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Learning Rate: 0.001, Batch Size: 16, Score: 0.08893406391143799
Best Score: 0.08893406391143799, Learning Rate: 0.001, Batch Size: 16


2024-01-18 00:21:06.554490: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-18 00:21:06.556543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-18 00:21:06.557591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - 1s 2ms/step
(87, 2)
(87, 2)
2


In [ ]:
from tensorflow.keras.backend import clear_session
import gc
clear_session()
del model
gc.collect()

In [243]:
import plotly.graph_objects as go
def visualize_future_distribution(results):
    '''
    Create stacked box and whisker plots for the predicted and real values
    '''

    fig = go.Figure()
    print(results.shape)

    for i in range(2):

        fig.add_trace(go.Box(y=results[f'predicted_{i+1}'], name=f'Predicted {i}')) 
        fig.add_trace(go.Box(y=results[f'real_{i+1}'], name=f'Real {i}'))

    fig.update_layout(
        title='Future Performance of Cluster',
        xaxis_title='Steps in future',
        yaxis_title='Cumulative Percent Change'
    ) 

    return fig

In [244]:
from plotly.subplots import make_subplots
bench_fig = visualize_future_distribution(results_benchmark)
tuned_fig = visualize_future_distribution(results_tuned)
fig = make_subplots(rows=1, cols=2)

for trace in bench_fig.data:
    fig.add_trace(trace, row=1, col=1)

for trace in tuned_fig.data:
    fig.add_trace(trace, row=1, col=2)

fig.show()

(87, 5)
(201, 13)


In [245]:
print(benchmarkAccuracy)
# print(val_loss_benchmark)
print(fineTunedAccuracy)
# print(val_loss_tuned)
# write results_tuned dataframe to csv
results_tuned.to_csv('results_tuned.csv')

Cluster Number:
Accuracy1D 47.13% PredictedRet: -0.004913469310849905 ActRet: 0.04711557244860369
Accuracy2D 37.93% PredictedRet: 0.008820226415991783 ActRet: 0.10982128947322142
Train set length: 314 Test set length: 87

Cluster Number:
Accuracy1D 55.22% PredictedRet: 0.015846332535147667 ActRet: 0.058342034458805876
Accuracy2D 61.19% PredictedRet: 0.043984852731227875 ActRet: 0.11842633418828696
Accuracy3D 59.2% PredictedRet: 0.07841123640537262 ActRet: 0.17888408636918932
Accuracy4D 57.21% PredictedRet: 0.11634019762277603 ActRet: 0.21772292785495792
Accuracy5D 60.2% PredictedRet: 0.15619394183158875 ActRet: 0.23187393435485146
Accuracy6D 61.19% PredictedRet: 0.19761110842227936 ActRet: 0.27124266650794554
Train set length: 707 Test set length: 201

